In [1]:
import cv2 as cv
import numpy as np
import face_recognition
import glob

In [14]:
i = cv.imread('Photos/others/cat.jpg')
cv.imshow('as', i)

In [37]:
elon_image1 = face_recognition.load_image_file('Faces/elon1.jpeg')
einstein_image1 = face_recognition.load_image_file('Faces/einstein1.jpeg')

elon_encoding1 = face_recognition.face_encodings(elon_image1)[0]
einstein_encoding1 = face_recognition.face_encodings(einstein_image1)[0]

#elon_image2 = face_recognition.load_image_file('Faces/elon2.webp')
einstein_image2 = face_recognition.load_image_file('Faces/einstein2.jpeg')

#elon_encoding2 = face_recognition.face_encodings(elon_image2)[0]
einstein_encoding2 = face_recognition.face_encodings(einstein_image2)[0]

results = face_recognition.compare_faces([einstein_encoding1], elon_encoding1)

In [38]:
results

[False]

In [15]:
#show webcam video
capture = cv.VideoCapture(0)
while True:
    isTrue, frame = capture.read()

    if isTrue:   
        frame = cv.flip(frame, 1)
        cv.imshow('Video', frame)
        if cv.waitKey(20) & 0xFF==ord('d'):
            break            
    else:
        break

capture.release()
cv.destroyAllWindows()

In [2]:
#face location recognition
capture = cv.VideoCapture(0)
process_this_frame = 0
storedFaceLocation = [(0,0,0,0)]
face_recog_scale_factor = 4

while True:
    isTrue, frame = capture.read()
    
    if cv.waitKey(20) & 0xFF==ord('q') or not(isTrue):
        break  
            
    frame = cv.flip(frame, 1)
    frame = cv.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2), interpolation = cv.INTER_CUBIC)
    process_this_frame-=1
    
    
    if process_this_frame <= 0:
        process_this_frame = 3
        small_frame = cv.resize(frame, (0, 0), fx=1 / face_recog_scale_factor, fy=1 / face_recog_scale_factor)
        rgb_frame = small_frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(rgb_frame)
        #face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        
        #for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        storedFaceLocation = face_locations
        
    for (top, right, bottom, left) in storedFaceLocation:
        cv.rectangle(frame, (left*face_recog_scale_factor, top*face_recog_scale_factor), (right*face_recog_scale_factor, bottom*face_recog_scale_factor), (0, 0, 255), 2)

    cv.imshow("Live", frame)

capture.release()
cv.destroyAllWindows()

In [3]:
#face and face landmarks 
capture = cv.VideoCapture(0)
process_this_frame = 0
storedFaceLocation = [(0,0,0,0)]
storedFeatureLocation = []
storedFPS = 0
face_recog_scale_factor = 4
previousTimestamp = cv.getTickCount()

while True:
    isTrue, frame = capture.read()
    
    if cv.waitKey(20) & 0xFF==ord('q') or not(isTrue):
        break  
            
    frame = cv.flip(frame, 1)
    frame = cv.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2), interpolation = cv.INTER_CUBIC)
    process_this_frame-=1
    
    
    if process_this_frame <= 0:
        process_this_frame = 3
        small_frame = cv.resize(frame, (0, 0), fx=1 / face_recog_scale_factor, fy=1 / face_recog_scale_factor)
        rgb_frame = small_frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(rgb_frame)
        face_landmarks_list = face_recognition.face_landmarks(rgb_frame)
        #face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        
        #for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        storedFaceLocation = face_locations
        
        storedFeatureLocation = []
        for face_landmarks in face_landmarks_list:
            for facial_feature in face_landmarks.keys():
                #print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))
                feature_points = np.array(face_landmarks[facial_feature])
                if len(feature_points) > 0:
                    with np.nditer(feature_points, op_flags=['readwrite']) as it:
                        for x in it:
                            x*=face_recog_scale_factor
                    storedFeatureLocation.append(feature_points)
                    
        tick = cv.getTickCount()
        t = (tick - previousTimestamp)/cv.getTickFrequency()
        previousTimestamp = tick
        storedFPS = 1 / t
        

                    

    for (top, right, bottom, left) in storedFaceLocation:
        cv.rectangle(frame, (left*face_recog_scale_factor, top*face_recog_scale_factor), (right*face_recog_scale_factor, bottom*face_recog_scale_factor), (0, 0, 255), 2)
    for featurePoints in storedFeatureLocation:
        cv.drawContours(frame, [featurePoints], 0, (255,255,255), 1)
    cv.putText(frame, "FPS:" + str("{:.2f}".format(storedFPS * 3)), (0,30), cv.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 1)
    cv.imshow("Live", frame)

capture.release()
cv.destroyAllWindows()

In [21]:
#checkerboard corner marker
CHECKERBOARD = (9,6)
capture = cv.VideoCapture(0)
process_this_frame = 0
scale_factor = 3

while True:
    isTrue, frame = capture.read()
    
    if cv.waitKey(20) & 0xFF==ord('q') or not(isTrue):
        break  

    frame = cv.flip(frame, 1)
    frame = cv.resize(frame, (frame.shape[1] // 2, frame.shape[0] // 2), interpolation = cv.INTER_CUBIC)
    process_this_frame-=1
    
    
    if process_this_frame <= 0:
        process_this_frame = 1
        small_frame = cv.resize(frame, (0, 0), fx=1 / scale_factor, fy=1 / scale_factor)
        gray = cv.cvtColor(small_frame,cv.COLOR_BGR2GRAY)
        ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD,None)
        if ret:
            for corner in corners:
                cv.drawMarker(frame, (int(corner[0][0])*scale_factor, int(corner[0][1])*scale_factor), (0, 255, 0), thickness=2)
        
    cv.imshow("Live", frame)

capture.release()
cv.destroyAllWindows()

In [16]:
#checkerboard axis or cube to corner
def drawHelper(f):
    x = int(f * scale_factor)
    if x > 100000:
        x = 100000
    if x < 0:
        x = 0
    return x

def draw(img, corners, imgpts):
    if drawType:
        return drawCube(img, corners, imgpts)
    else:
        return drawAxis(img, corners, imgpts)

def drawAxis(img, corners, imgpts):
    corner = tuple(map(drawHelper, corners[0].ravel()))
    img = cv.line(img, corner, tuple(map(drawHelper, imgpts[0].ravel())), (255,0,0), 5)
    img = cv.line(img, corner, tuple(map(drawHelper, imgpts[1].ravel())), (0,255,0), 5)
    img = cv.line(img, corner, tuple(map(drawHelper, imgpts[2].ravel())), (0,0,255), 5)
    return img

def drawCube(img, corners, imgpts):
    imgpts = np.array([[drawHelper(f) for f in p] for p in np.int32(imgpts).reshape(-1,2)])
    # draw ground floor in green
    img = cv.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

        
capture = cv.VideoCapture(0)
process_this_frame = 0
screen_scale_factor = 2
scale_factor = 2
max_cali_frame_size = 1
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
use_stored_cali_matrix = 1
drawType = 1

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((1, CHECKERBOARD[0]* CHECKERBOARD[1],3), np.float32)
objp[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],0:CHECKERBOARD[1]].T.reshape(-1, 2)

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

if drawType:
    axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])
    
    


while True:
    isTrue, frame = capture.read()
    
    if cv.waitKey(20) & 0xFF==ord('q') or not(isTrue):
        break  

    frame = cv.flip(frame, 1)
    frame = cv.resize(frame, (frame.shape[1]//screen_scale_factor, frame.shape[0]//screen_scale_factor), interpolation = cv.INTER_CUBIC)
    process_this_frame-=1
    
    
    
    if process_this_frame <= 0:
        process_this_frame = 1
        small_frame = cv.resize(frame, (0, 0), fx=1 / scale_factor, fy=1 / scale_factor)
        gray = cv.cvtColor(small_frame,cv.COLOR_BGR2GRAY)
        ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD,None)
        

        if ret:
            if len(objpoints) >= max_cali_frame_size:
                objpoints.pop(0)
                imgpoints.pop(0)

            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners)
            
            ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
#             print(mtx)
            if use_stored_cali_matrix:
                mtx = np.array([[272.59322508,0,162.70908309],[0,272.00736526,86.05319646],[0,0,1]])
            ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
            imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
            frame = draw(frame,corners2,imgpts)
            
        
    cv.imshow("Live", frame)

capture.release()
cv.destroyAllWindows()

In [29]:
stored_mtx

array([[161.65787951,   0.        , 178.40852048],
       [  0.        ,  84.08703535,  45.57696538],
       [  0.        ,   0.        ,   1.        ]])

In [8]:
#calibration 
CHECKERBOARD = (9,6)
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((1, CHECKERBOARD[0]* CHECKERBOARD[1],3), np.float32)
objp[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],0:CHECKERBOARD[1]].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('Photos/random/*')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, None)
    # If found, add object points, image points (after refining them)
    print(ret)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        #cv.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        #cv.imshow('img', img)
        #cv.waitKey(200)
#cv.destroyAllWindows()
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

True


In [23]:
# calibration 2
img = cv.imread('Photos/calib_sample/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))


array([[536.39560391,   0.        , 334.03199451],
       [  0.        , 536.43966752, 239.43394385],
       [  0.        ,   0.        ,   1.        ]])

NameError: name 'img' is not defined

In [12]:
# draw axis on corner
def draw(img, corners, imgpts):
    corner = tuple(map(int, corners[0].ravel()))
    img = cv.line(img, corner, tuple(map(int, imgpts[0].ravel())), (255,0,0), 5)
    img = cv.line(img, corner, tuple(map(int, imgpts[1].ravel())), (0,255,0), 5)
    img = cv.line(img, corner, tuple(map(int, imgpts[2].ravel())), (0,0,255), 5)
    return img

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((1, CHECKERBOARD[0]* CHECKERBOARD[1],3), np.float32)
objp[0, :, :2] = np.mgrid[0:CHECKERBOARD[0],0:CHECKERBOARD[1]].T.reshape(-1, 2)
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

for fname in glob.glob('Photos/random/*'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD,None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
        # project 3D points to image plane
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
        img = draw(img,corners2,imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0) & 0xFF
        if k == ord('s'):
            cv.imwrite(fname[:6]+'.png', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
img = cv.imread('Photos/calib_sample/left12.jpg')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, corners = cv.findChessboardCorners(gray, (7,6),None)
for corner in corners:
    cv.drawMarker(img, (int(corner[0][0]), int(corner[0][1])), (0, 255, 0), thickness=2)
cv.imshow('img', img)
cv.waitKey(0)

In [6]:
matrix

array([[20.10652776,  0.        , 84.16362008],
       [ 0.        , 20.34237943, 95.42267758],
       [ 0.        ,  0.        ,  1.        ]])